## [Issue #7](https://github.com/yeemey/haackwell/issues/7)

### Map pattern of polluted, damaged, stolen, etc.

1. Split out free text data from 'STATUS' column into more specific columns.
2. Apply polluted etc. trends to map.

#### Questions

1. Are mechanical pump failures/pollution related to pump type/make?

In [3]:
import pandas as pd
import re
import matplotlib
import matplotlib.pyplot as plt

In [4]:
# Water Point Data Exchange
wpdx_data = 'https://www.hydroshare.org/django_irods/download/d3659dcf575d4090801a74d1ce096d7c/data/contents/WPDx_Well_Function_Upd_151224_xy161117.csv'
wpdx_df = pd.read_csv(wpdx_data)

In [5]:
wpdx_df

,WELL_ID,LAT_DD,LONG_DD,FUNC,STATUS,COD_FCN,COD_QTY,COD_RESRCE,ADM1,ADM2,ACTIVITY,COUNTRY,WATERSRC,WATERTECH,INSTALLED,MGMT,PAY,SOURCE,RPT_DATE
0,362092,5.982436,-8.180609,Yes,Working but with problems. Well polluted|Under...,2,1,0,Grand Gedeh,Tchien,zmbpw,LR,Manual pump on hand-dug well,Vergnet,0,NaN,No water committee,WASH Liberia,21/01/2011
1,362100,5.899207,-8.173315,Yes,Working but with problems. Well polluted|Under...,2,1,0,Grand Gedeh,Tchien,z4ja5,LR,Manual pump on hand-dug well,Vergnet,1986,NaN,No water committee,WASH Liberia,21/01/2011
2,357349,5.802157,-9.645714,Yes,Working but with problems. Not priming,2,1,0,Rivercess,Norwein,yjryl,LR,Manual pump on hand-dug well,Afridev,2008,NaN,No water committee,WASH Liberia,02/02/2011
3,489514,-0.541100,34.375820,No,Drought|No operation in the dry season,999,0,1,Homa Bay,Mbita,Yao Oinga,KE,NaN,Surface water,0,0,No payment system,Engineering Sciences & Global Development,24/01/2011
4,357595,5.716055,-9.618187,No,Broken Down System. low water table,0,1,1,Rivercess,Norwein,y1ccy,LR,Manual pump on hand-dug well,Afridev,2005,NaN,No water committee,WASH Liberia,03/02/2011
5,489266,-0.730000,34.366000,No,No fuel|No operation at least once a week,2,1,0,Homa Bay,Ndhiwa,Water Kiosk,KE,NaN,Gravity-fed communal standpipe,1995,Private Operator/Delegated Management,Per Bucket,Engineering Sciences & Global Development,18/02/2011
6,489625,-0.450333,34.009880,No,Low yield|No operation in the dry season,2,1,1,Homa Bay,Mbita,Wakula Dispensary BH,KE,NaN,Borehole with hand pump,0,Institutional Management,No payment system,Engineering Sciences & Global Development,02/10/2011
7,364570,5.231378,-9.141873,Yes,Working but with problems. Well polluted|Under...,2,1,0,Sinoe,Sanquin Dist#2,v5dph,LR,Manual pump on hand-dug well,Afridev,2010,Community Management,Yes but only in case of breakdown,WASH Liberia,27/01/2011
8,361779,5.225134,-8.121493,Yes,Working but with problems. insufficient water,2,1,0,River Gee,Karforh,v42mh,LR,Manual pump on hand-dug well,Afridev,2009,NaN,No water committee,WASH Liberia,18/02/2011
9,361780,5.225422,-8.119787,Yes,Working but with problems. insufficient water,2,1,0,River Gee,Karforh,v42mf,LR,Manual pump on hand-dug well,Afridev,2009,NaN,No water committee,WASH Liberia,18/02/2011


In [7]:
well_status_df = wpdx_df[['WELL_ID', 'LAT_DD', 'LONG_DD', 'FUNC', 'STATUS', 'WATERSRC', 'WATERTECH']].copy()
well_status_df

,WELL_ID,LAT_DD,LONG_DD,FUNC,STATUS,WATERSRC,WATERTECH
0,362092,5.982436,-8.180609,Yes,Working but with problems. Well polluted|Under...,Manual pump on hand-dug well,Vergnet
1,362100,5.899207,-8.173315,Yes,Working but with problems. Well polluted|Under...,Manual pump on hand-dug well,Vergnet
2,357349,5.802157,-9.645714,Yes,Working but with problems. Not priming,Manual pump on hand-dug well,Afridev
3,489514,-0.541100,34.375820,No,Drought|No operation in the dry season,NaN,Surface water
4,357595,5.716055,-9.618187,No,Broken Down System. low water table,Manual pump on hand-dug well,Afridev
5,489266,-0.730000,34.366000,No,No fuel|No operation at least once a week,NaN,Gravity-fed communal standpipe
6,489625,-0.450333,34.009880,No,Low yield|No operation in the dry season,NaN,Borehole with hand pump
7,364570,5.231378,-9.141873,Yes,Working but with problems. Well polluted|Under...,Manual pump on hand-dug well,Afridev
8,361779,5.225134,-8.121493,Yes,Working but with problems. insufficient water,Manual pump on hand-dug well,Afridev
9,361780,5.225422,-8.119787,Yes,Working but with problems. insufficient water,Manual pump on hand-dug well,Afridev


In [8]:
well_status_df['STATUS'].value_counts()

DRY                                                                                                                                               3708
Status:Functional|Quantity:Insufficient|Quality:Soft                                                                                              3323
Functional ( in use)                                                                                                                              2971
Status:Not functional|Quantity:Insufficient|Quality:Soft                                                                                          1572
Status:Functional|Quantity:Seasonal|Quality:Soft                                                                                                   394
Status:Not functional|Quantity:Dry|Quality:Soft                                                                                                    305
Status:Functional|Quantity:Insufficient|Quality:Salty                                         

In [9]:
well_status_df['WATERTECH'].value_counts()

DUG WELL                                   3535
Gravity Communal standpipe                 1988
Nira/Tanira Hand pump                      1364
Hand Pump|Afridev                           946
Gravity                                     596
Mono Communal standpipe multiple            555
Protected spring                            508
Gravity Communal standpipe multiple         461
Mono Communal standpipe                     460
Submersible pump|Electricity                412
SWN 80 Hand pump                            394
None Hand pump                              366
Shallow well                                355
Deep borehole                               342
Afridev Hand pump                           278
None Other                                  239
Gravity Communal stand pipe                 234
Submersible Communal standpipe multiple     234
Hand manual                                 203
None Communal standpipe                     203
Gravity Improved spring                 

In [47]:
well_status = well_status_df['STATUS'].tolist()
nonfunc_status = []
func_status = []
for status in well_status:
    if re.search('Status:Not functional', status):
        nonfunc_status.append(status.split('|'))
    #elif re.search('Non-Functional - ', status):
        #nonfunc_status.append(status.split(' - '))
    elif re.search('Status:Functional', status):
        func_status.append(status.split('|'))
    #elif re.search('Functional - ', status):
        #func_status.append(status.split(' - '))

In [48]:
print(nonfunc_status)

[['Status:Not functional', 'Breakdown Year:1961', 'Quantity:Insufficient', 'Quality:Soft'], ['Status:Not functional', 'Breakdown Year:1967', 'Quantity:Insufficient', 'Quality:Soft'], ['Status:Not functional', 'Breakdown Year:1970', 'Reason Not Functioning:Abandoned', 'Quantity:Dry', 'Quality:Soft'], ['Status:Not functional', 'Breakdown Year:1975', 'Quantity:Insufficient', 'Quality:Soft'], ['Status:Not functional', 'Breakdown Year:1975', 'Quantity:Seasonal', 'Quality:Soft'], ['Status:Not functional', 'Breakdown Year:1976', 'Quantity:Enough', 'Quality:Soft'], ['Status:Not functional', 'Breakdown Year:1976', 'Quantity:Enough', 'Quality:Soft'], ['Status:Not functional', 'Breakdown Year:1976', 'Quantity:Enough', 'Quality:Soft'], ['Status:Not functional', 'Breakdown Year:1976', 'Quantity:Enough', 'Quality:Soft'], ['Status:Not functional', 'Breakdown Year:1976', 'Quantity:Enough', 'Quality:Soft'], ['Status:Not functional', 'Breakdown Year:1976', 'Quantity:Enough', 'Quality:Soft'], ['Status:No

In [49]:
print(func_status)

[['Status:Functional', 'Breakdown Year:1978', 'Reason Not Functioning:Abandoned pumping scheme', 'Quantity:Dry', 'Quality:Soft'], ['Status:Functional', 'Breakdown Year:1978', 'Reason Not Functioning:Replacement of pump machine', 'Quantity:Insufficient', 'Quality:Soft'], ['Status:Functional', 'Breakdown Year:1978', 'Reason Not Functioning:Replacement of pump machine', 'Quantity:Insufficient', 'Quality:Soft'], ['Status:Functional', 'Breakdown Year:1978', 'Reason Not Functioning:Replacement of pump machine', 'Quantity:Insufficient', 'Quality:Soft'], ['Status:Functional', 'Breakdown Year:1978', 'Reason Not Functioning:Replacement of pump machine', 'Quantity:Insufficient', 'Quality:Soft'], ['Status:Functional', 'Breakdown Year:1978', 'Reason Not Functioning:Replacement of pump machine', 'Quantity:Insufficient', 'Quality:Soft'], ['Status:Functional', 'Breakdown Year:1978', 'Reason Not Functioning:Replacement of pump machine', 'Quantity:Insufficient', 'Quality:Soft'], ['Status:Functional', 'B